In [1]:
import pyspark.sql.functions as funct
from pyspark.sql.functions import broadcast
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, StringType

In [2]:
r=spark.createDataFrame([("A","B"),("B","C"),("B","D"),("D","E"),("F","G"),("G","H")]).toDF("N1","N2") 
r.show()
#r.printSchema()

+---+---+
 N1| N2|
+---+---+
 A| B|
 B| C|
 B| D|
 D| E|
 F| G|
 G| H|
+---+---+

In [3]:
g =  spark.read\
            .format("csv").option("header", "true")\
            .option("inferSchema", "true")\
            .load("/FileStore/tables/googleS-3.csv")
g.show(10)

+------+------+
 N1| N2|
+------+------+
 0| 11342|
 0|824020|
 0|867923|
 0|891835|
 11342| 0|
 11342| 27469|
867923|760842|
867923|815602|
867923|891835|
891835| 0|
+------+------+
only showing top 10 rows

In [4]:
new_pair_flag = True
iteration = 0
dedupJob = g
accum = sc.accumulator(0)
dedupJob.show()

+------+------+
 N1| N2|
+------+------+
 0| 11342|
 0|824020|
 0|867923|
 0|891835|
 11342| 0|
 11342| 27469|
867923|760842|
867923|815602|
867923|891835|
891835| 0|
891835| 11342|
891835|112028|
891835|235849|
891835|857527|
891835|867923|
 1| 53051|
 1|203402|
 1|223236|
 1|276233|
 1|552600|
+------+------+
only showing top 20 rows

In [5]:
#functions without secondary sort
def min_reduce(N1,N2s):
    min = N1
    for value in N2s:
        if value < min:
            min = value
    return min
udf_min_reduce = funct.udf(min_reduce)

def suite_reduce(N1,Min,N2):
  #  global newPairs
    if Min==N2: #could cause error?
      return N1
    else:
     # newPairs += 1
      accum.add(1)
      return N2
udf_suite_reduce = funct.udf(suite_reduce)

In [6]:
# test
def suite_reduce2(N1,Min,N2):
  #  global newPairs
    if Min < N2:
      #accum.add(1)
      return N2
    else:
     # newPairs += 1
      return N1
udf_suite_reduce2 = funct.udf(lambda x,y,z: suite_reduce2(x,y,z),StringType())
_=spark.udf.register("udf_suite_reduce2", suite_reduce2)

In [7]:
#secondary sort
def CCF_reduce_SS_min(N1, N2s):
  Min=N2s[0]
  if N1 < Min:
    Min=N1
  return Min
udf_CCF_reduce_SS_min = funct.udf(lambda x,y: CCF_reduce_SS_min(x,y),StringType())
_=spark.udf.register("udf_CCF_reduce_SS_min", CCF_reduce_SS_min)

In [8]:
#Unit test with Secondary sort

In [9]:
mapJob = dedupJob.union(dedupJob.select('N2', 'N1'))
mapJob.count()

Out[31]: 80

In [10]:
reduceJob=mapJob.groupby("N1").agg(funct.sort_array(funct.collect_set("N2")).alias('N2s'))
reduceJob.show(100, truncate=False)

+------+-------------------------------------------------------------------------------+
N1 |N2s |
+------+-------------------------------------------------------------------------------+
0 |[11342, 824020, 867923, 891835] |
11342 |[0, 27469, 891835] |
867923|[0, 760842, 815602, 891835] |
891835|[0, 11342, 112028, 235849, 857527, 867923] |
1 |[53051, 203402, 223236, 276233, 552600, 569212, 635575, 748615, 862566, 893884]|
203402|[1, 53051, 164684, 184896, 184984, 223236] |
795253|[746182] |
5 |[39733, 219773, 300279, 535142] |
300279|[5, 535142, 701345, 749671] |
535142|[5, 39733, 300279] |
824020|[0] |
27469 |[11342] |
760842|[867923] |
815602|[867923] |
112028|[891835] |
235849|[891835] |
857527|[891835] |
53051 |[1, 203402] |
223236|[1, 203402] |
276233|[1] |
552600|[1] |
569212|[1] |
635575|[1] |
748615|[1] |
862566|[1] |
893884|[1] |
164684|[203402] |
184896|[203402] |
184984|[203402] |
746182|[795253] |
39733 |[5, 535142] |
219773|[5] |
701345|[300279] |
749671|[300279] |
+------+-------------------------------------------------------------------------------+

In [11]:
R2=reduceJob.withColumn('MinN',udf_CCF_reduce_SS_min('N1','N2s'))
R2.show(100,truncate=False)

+------+-------------------------------------------------------------------------------+------+
N1 |N2s |MinN |
+------+-------------------------------------------------------------------------------+------+
0 |[11342, 824020, 867923, 891835] |0 |
11342 |[0, 27469, 891835] |0 |
867923|[0, 760842, 815602, 891835] |0 |
891835|[0, 11342, 112028, 235849, 857527, 867923] |0 |
1 |[53051, 203402, 223236, 276233, 552600, 569212, 635575, 748615, 862566, 893884]|1 |
203402|[1, 53051, 164684, 184896, 184984, 223236] |1 |
795253|[746182] |746182|
5 |[39733, 219773, 300279, 535142] |5 |
300279|[5, 535142, 701345, 749671] |5 |
535142|[5, 39733, 300279] |5 |
824020|[0] |0 |
27469 |[11342] |11342 |
760842|[867923] |760842|
815602|[867923] |815602|
112028|[891835] |112028|
235849|[891835] |235849|
857527|[891835] |857527|
53051 |[1, 203402] |1 |
223236|[1, 203402] |1 |
276233|[1] |1 |
552600|[1] |1 |
569212|[1] |1 |
635575|[1] |1 |
748615|[1] |1 |
862566|[1] |1 |
893884|[1] |1 |
164684|[203402] |164684|
184896|[203402] |184896|
184984|[203402] |184984|
746182|[795253] |746182|
39733 |[5, 535142] |5 |
219773|[5] |5 |
701345|[300279] |300279|
749671|[300279] |300279|
+------+-------------------------------------------------------------------------------+------+

In [12]:
R3=R2.withColumn('N2',funct.explode("N2s"))
R3.show(100,truncate=False)

+------+------------------------------------------+------+------+
N1 |N2s |MinN |N2 |
+------+------------------------------------------+------+------+
11342 |[0, 27469, 891835] |0 |0 |
11342 |[0, 27469, 891835] |0 |27469 |
11342 |[0, 27469, 891835] |0 |891835|
867923|[0, 760842, 891835, 815602] |0 |0 |
867923|[0, 760842, 891835, 815602] |0 |760842|
867923|[0, 760842, 891835, 815602] |0 |891835|
867923|[0, 760842, 891835, 815602] |0 |815602|
891835|[0, 857527, 112028, 867923, 235849, 11342]|0 |0 |
891835|[0, 857527, 112028, 867923, 235849, 11342]|0 |857527|
891835|[0, 857527, 112028, 867923, 235849, 11342]|0 |112028|
891835|[0, 857527, 112028, 867923, 235849, 11342]|0 |867923|
891835|[0, 857527, 112028, 867923, 235849, 11342]|0 |235849|
891835|[0, 857527, 112028, 867923, 235849, 11342]|0 |11342 |
203402|[164684, 53051, 1, 184896, 223236, 184984]|1 |164684|
203402|[164684, 53051, 1, 184896, 223236, 184984]|1 |53051 |
203402|[164684, 53051, 1, 184896, 223236, 184984]|1 |1 |
203402|[164684, 53051, 1, 184896, 223236, 184984]|1 |184896|
203402|[164684, 53051, 1, 184896, 223236, 184984]|1 |223236|
203402|[164684, 53051, 1, 184896, 223236, 184984]|1 |184984|
795253|[746182] |746182|746182|
300279|[5, 701345, 535142, 749671] |5 |5 |
300279|[5, 701345, 535142, 749671] |5 |701345|
300279|[5, 701345, 535142, 749671] |5 |535142|
300279|[5, 701345, 535142, 749671] |5 |749671|
535142|[5, 39733, 300279] |5 |5 |
535142|[5, 39733, 300279] |5 |39733 |
535142|[5, 39733, 300279] |5 |300279|
824020|[0] |0 |0 |
27469 |[11342] |11342 |11342 |
53051 |[1, 203402] |1 |1 |
53051 |[1, 203402] |1 |203402|
223236|[1, 203402] |1 |1 |
223236|[1, 203402] |1 |203402|
276233|[1] |1 |1 |
552600|[1] |1 |1 |
569212|[1] |1 |1 |
635575|[1] |1 |1 |
748615|[1] |1 |1 |
862566|[1] |1 |1 |
893884|[1] |1 |1 |
39733 |[5, 535142] |5 |5 |
39733 |[5, 535142] |5 |535142|
219773|[5] |5 |5 |
701345|[300279] |300279|300279|
749671|[300279] |300279|300279|
+------+------------------------------------------+------+------+

In [13]:
R4= R3.withColumn('NewN1',udf_suite_reduce('N1','MinN','N2'))
R4.show(100,truncate=False)

+------+------------------------------------------+------+------+------+
N1 |N2s |MinN |N2 |NewN1 |
+------+------------------------------------------+------+------+------+
11342 |[0, 27469, 891835] |0 |0 |0 |
11342 |[0, 27469, 891835] |0 |27469 |27469 |
11342 |[0, 27469, 891835] |0 |891835|891835|
867923|[0, 760842, 891835, 815602] |0 |0 |0 |
867923|[0, 760842, 891835, 815602] |0 |760842|760842|
867923|[0, 760842, 891835, 815602] |0 |891835|891835|
867923|[0, 760842, 891835, 815602] |0 |815602|815602|
891835|[0, 857527, 112028, 867923, 235849, 11342]|0 |0 |0 |
891835|[0, 857527, 112028, 867923, 235849, 11342]|0 |857527|857527|
891835|[0, 857527, 112028, 867923, 235849, 11342]|0 |112028|112028|
891835|[0, 857527, 112028, 867923, 235849, 11342]|0 |867923|867923|
891835|[0, 857527, 112028, 867923, 235849, 11342]|0 |235849|235849|
891835|[0, 857527, 112028, 867923, 235849, 11342]|0 |11342 |11342 |
203402|[164684, 53051, 1, 184896, 223236, 184984]|1 |164684|164684|
203402|[164684, 53051, 1, 184896, 223236, 184984]|1 |53051 |53051 |
203402|[164684, 53051, 1, 184896, 223236, 184984]|1 |1 |1 |
203402|[164684, 53051, 1, 184896, 223236, 184984]|1 |184896|184896|
203402|[164684, 53051, 1, 184896, 223236, 184984]|1 |223236|223236|
203402|[164684, 53051, 1, 184896, 223236, 184984]|1 |184984|184984|
795253|[746182] |746182|746182|746182|
300279|[5, 701345, 535142, 749671] |5 |5 |5 |
300279|[5, 701345, 535142, 749671] |5 |701345|701345|
300279|[5, 701345, 535142, 749671] |5 |535142|535142|
300279|[5, 701345, 535142, 749671] |5 |749671|749671|
535142|[5, 39733, 300279] |5 |5 |5 |
535142|[5, 39733, 300279] |5 |39733 |39733 |
535142|[5, 39733, 300279] |5 |300279|300279|
824020|[0] |0 |0 |0 |
27469 |[11342] |11342 |11342 |11342 |
53051 |[1, 203402] |1 |1 |1 |
53051 |[1, 203402] |1 |203402|203402|
223236|[1, 203402] |1 |1 |1 |
223236|[1, 203402] |1 |203402|203402|
276233|[1] |1 |1 |1 |
552600|[1] |1 |1 |1 |
569212|[1] |1 |1 |1 |
635575|[1] |1 |1 |1 |
748615|[1] |1 |1 |1 |
862566|[1] |1 |1 |1 |
893884|[1] |1 |1 |1 |
39733 |[5, 535142] |5 |5 |5 |
39733 |[5, 535142] |5 |535142|535142|
219773|[5] |5 |5 |5 |
701345|[300279] |300279|300279|300279|
749671|[300279] |300279|300279|300279|
+------+------------------------------------------+------+------+------+

In [14]:
sqlContext.setConf("spark.sql.shuffle.partitions", "1")

In [15]:
#without Secondary sort
while new_pair_flag:
    iteration += 1
    #newPair = False
    #accum.value = 0

    # CCF-iterate (MAP)
    mapJob = dedupJob.union(dedupJob.select('N2', 'N1'))#.persist(storage_level)

    # CCF-iterate (REDUCE)
    
    R_min=mapJob.groupby("N1").agg(funct.collect_set("N2").alias('N2s'))\
    .withColumn('N2',udf_min_reduce('N1','N2s')).where('N2<N1').persist()
    R_reduce_part1=R_min.select('N1','N2')
    R_reduce_part2=R_min.withColumn('NewN1',funct.explode("N2s")).where('NewN1<>N2')\
    .select('NewN1','N2').withColumnRenamed('NewN1','N1')
    reduceJob=R_reduce_part1.union(R_reduce_part2)
    
    # CCF-dedup 
    dedupJob = reduceJob.distinct()#.persist(storage_level) #.orderBy("N1", "N2")
    # Force the RDD evalusation
    tmp = dedupJob.count()

    # Prepare next iteration
    #mapJob = dedupJob
    #new_pair_flag = bool(accum.value)#newPairs)
    new_pair_flag=R_reduce_part2.count()

    #LOGGER.warn("Iteration :"+str(iteration)+" --- "+" newPair : "+str(new_pair_flag))
    print("Iteration :"+str(iteration)+" --- "+" newPairs : "+ str(new_pair_flag))#newPairs))
    print(dedupJob.groupby("N2").agg(funct.sort_array(funct.collect_set("N1")).alias('N1s')).show(200,truncate=False))

Iteration :1 --- newPairs : 23
+------+-------------------------------------------------------------------------------------------------------+
N2 |N1s |
+------+-------------------------------------------------------------------------------------------------------+
0 |[11342, 27469, 112028, 235849, 760842, 815602, 824020, 857527, 867923, 891835] |
1 |[53051, 164684, 184896, 184984, 203402, 223236, 276233, 552600, 569212, 635575, 748615, 862566, 893884]|
746182|[795253] |
5 |[39733, 219773, 300279, 535142, 701345, 749671] |
11342 |[27469] |
300279|[701345, 749671] |
+------+-------------------------------------------------------------------------------------------------------+

None
Iteration :2 --- newPairs : 9
+------+-------------------------------------------------------------------------------------------------------+
N2 |N1s |
+------+-------------------------------------------------------------------------------------------------------+
0 |[112028, 11342, 235849, 27469, 760842, 815602, 824020, 857527, 867923, 891835] |
1 |[164684, 184896, 184984, 203402, 223236, 276233, 53051, 552600, 569212, 635575, 748615, 862566, 893884]|
746182|[795253] |
5 |[535142] |
300279|[5, 701345, 749671] |
219773|[300279, 39733, 5, 535142, 701345, 749671] |
+------+-------------------------------------------------------------------------------------------------------+

None
Iteration :3 --- newPairs : 8
+------+-------------------------------------------------------------------------------------------------------+
N2 |N1s |
+------+-------------------------------------------------------------------------------------------------------+
0 |[112028, 11342, 235849, 27469, 760842, 815602, 824020, 857527, 867923, 891835] |
1 |[164684, 184896, 184984, 203402, 223236, 276233, 53051, 552600, 569212, 635575, 748615, 862566, 893884]|
746182|[795253] |
219773|[300279, 39733, 5, 535142, 701345, 749671] |
+------+-------------------------------------------------------------------------------------------------------+

None
Iteration :4 --- newPairs : 0
+------+-------------------------------------------------------------------------------------------------------+
N2 |N1s |
+------+-------------------------------------------------------------------------------------------------------+
0 |[112028, 11342, 235849, 27469, 760842, 815602, 824020, 857527, 867923, 891835] |
1 |[164684, 184896, 184984, 203402, 223236, 276233, 53051, 552600, 569212, 635575, 748615, 862566, 893884]|
746182|[795253] |
219773|[300279, 39733, 5, 535142, 701345, 749671] |
+------+-------------------------------------------------------------------------------------------------------+

None

In [16]:
#with secondary sort
while new_pair_flag:
    iteration += 1
    #newPair = False
    #accum.value = 0

    # CCF-iterate (MAP)
    mapJob = dedupJob.union(dedupJob.select('N2', 'N1'))#.persist(storage_level)

    # CCF-iterate (REDUCE)
    #with secondary sort
    #R_min=mapJob.groupby("N1").agg(funct.collect_set("N2").alias('N2s'))\
    #.withColumn('N2',udf_min_reduce('N1','N2s')).where('N2<N1').persist()
    R_min=mapJob.groupby("N1").agg(funct.sort_array(funct.collect_set("N2")).alias('N2s'))\
    .withColumn('N2',udf_CCF_reduce_SS_min('N1','N2s')).where('N2<N1').persist()

    R_reduce_part1=R_min.select('N1','N2')
    R_reduce_part2=R_min.withColumn('NewN1',funct.explode("N2s")).where('NewN1<>N2')\
    .select('NewN1','N2').withColumnRenamed('NewN1','N1')
    reduceJob=R_reduce_part1.union(R_reduce_part2)
    
    # CCF-dedup 
    dedupJob = reduceJob.distinct()#.persist(storage_level) #.orderBy("N1", "N2")
    # Force the RDD evalusation
    tmp = dedupJob.count()

    # Prepare next iteration
    #mapJob = dedupJob
    #new_pair_flag = bool(accum.value)#newPairs)
    new_pair_flag=R_reduce_part2.count()

    #LOGGER.warn("Iteration :"+str(iteration)+" --- "+" newPair : "+str(new_pair_flag))
    print("Iteration :"+str(iteration)+" --- "+" newPairs : "+ str(new_pair_flag))#newPairs))
    print(dedupJob.groupby("N2").agg(funct.sort_array(funct.collect_set("N1")).alias('N1s')).show(200,truncate=False))

Iteration :1 --- newPairs : 23
+------+-------------------------------------------------------------------------------------------------------+
N2 |N1s |
+------+-------------------------------------------------------------------------------------------------------+
0 |[11342, 27469, 112028, 235849, 760842, 815602, 824020, 857527, 867923, 891835] |
1 |[53051, 164684, 184896, 184984, 203402, 223236, 276233, 552600, 569212, 635575, 748615, 862566, 893884]|
746182|[795253] |
5 |[39733, 219773, 300279, 535142, 701345, 749671] |
11342 |[27469] |
300279|[701345, 749671] |
+------+-------------------------------------------------------------------------------------------------------+

None
Iteration :2 --- newPairs : 9
+------+-------------------------------------------------------------------------------------------------------+
N2 |N1s |
+------+-------------------------------------------------------------------------------------------------------+
0 |[112028, 11342, 235849, 27469, 760842, 815602, 824020, 857527, 867923, 891835] |
1 |[164684, 184896, 184984, 203402, 223236, 276233, 53051, 552600, 569212, 635575, 748615, 862566, 893884]|
746182|[795253] |
5 |[535142] |
300279|[5, 701345, 749671] |
219773|[300279, 39733, 5, 535142, 701345, 749671] |
+------+-------------------------------------------------------------------------------------------------------+

None
Iteration :3 --- newPairs : 8
+------+-------------------------------------------------------------------------------------------------------+
N2 |N1s |
+------+-------------------------------------------------------------------------------------------------------+
0 |[112028, 11342, 235849, 27469, 760842, 815602, 824020, 857527, 867923, 891835] |
1 |[164684, 184896, 184984, 203402, 223236, 276233, 53051, 552600, 569212, 635575, 748615, 862566, 893884]|
746182|[795253] |
219773|[300279, 39733, 5, 535142, 701345, 749671] |
+------+-------------------------------------------------------------------------------------------------------+

None
Iteration :4 --- newPairs : 0
+------+-------------------------------------------------------------------------------------------------------+
N2 |N1s |
+------+-------------------------------------------------------------------------------------------------------+
0 |[112028, 11342, 235849, 27469, 760842, 815602, 824020, 857527, 867923, 891835] |
1 |[164684, 184896, 184984, 203402, 223236, 276233, 53051, 552600, 569212, 635575, 748615, 862566, 893884]|
746182|[795253] |
219773|[300279, 39733, 5, 535142, 701345, 749671] |
+------+-------------------------------------------------------------------------------------------------------+

None